In [2]:
import pandas as pd
import numpy as np

Bad val 'Flase' on line #329
	"axes.unicode_minus  : Flase    ## use unicode for the minus symbol
"
	in file "/Users/louis/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc"
	Key axes.unicode_minus: Could not convert "flase" to boolean


In [3]:
df = pd.read_excel('../OpenData/2020中華職棒上半季-電子紀錄程式檔-revised-20200925.xlsx',sheet_name='工作表1')

In [4]:
pitchData = df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,29,30,31,32,42,43,44,51,52]]
#pitchData.head()

In [5]:
# 切表，一個投手一張表
pitcher = [pitchData.loc[pitchData.投手 == x] for x in pitchData['投手'].drop_duplicates()]

In [6]:
# 算好壞球比與總球數
pitchCount = pd.DataFrame(columns=['name','total','strike','ball','rate'])
count = 0
for i in pitcher:
    s = 0
    b = 0
    for j in i['好壞球']:
        if j == '好球':
            s += 1
        elif j == '壞球':
            b += 1
    if b !=0 :
        rate = s / b
    else:
        rate = 1
    #print('name:',i.iloc[0,7],'total:',s+b,'strike:',s,'ball:',b,'rate:',rate)
    
    pitchCount.loc[count] = [i.iloc[0,7],s+b,s,b,rate]
    count += 1
        

In [7]:
# 好壞球平均值、max min計算
for i in pitchCount.iloc[:,1:]:
    a = pitchCount[i].agg(['max','min','mean'])
    print(i,'\n',a,'\n')

total 
 max     1591.00000
min        1.00000
mean     451.60241
Name: total, dtype: float64 

strike 
 max     1022.000000
min        1.000000
mean     288.084337
Name: strike, dtype: float64 

ball 
 max     577.000000
min       0.000000
mean    163.518072
Name: ball, dtype: float64 

rate 
 max     4.000000
min     0.700000
mean    1.700488
Name: rate, dtype: float64 



In [8]:
# 算總球數、好壞球比
pitchCount = pitchCount[pitchCount.total >= 500].sort_values(by='rate',ascending=False)
pitchCount.set_index('name',inplace = True)

In [9]:
# 算總球數、好壞球比
pitchCount[pitchCount.total >= 500].sort_values(by='total',ascending=False)

,total,strike,ball,rate
name,,,,
兄弟米蘭達,1591,1014,577,1.757366
富邦索沙,1496,1022,474,2.156118
兄弟德保拉,1469,922,547,1.685558
樂天霸能,1367,863,504,1.712302
樂天卡本特,1272,846,426,1.985915
兄弟黃恩賜,1219,817,402,2.032338
富邦伍鐸,1199,775,424,1.827830
樂天王溢正,1056,676,380,1.778947
富邦陳仕朋,1019,661,358,1.846369


In [10]:
#[[x] for x in len(pitchData)]
pitchData['局數'].fillna(method='ffill',inplace=True)
a = [str(int(pitchData['局數'][x])) + pitchData['Inning Half'][x] for x in range(len(pitchData))]
pitchData['innAndHalf'] = a

/Users/louis/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/louis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [68]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# 先切表，變成一局一局的 。要把壘上跑者 往下移一格
game = [pitchData.loc[pitchData.game_id == x] for x in pitchData['game_id'].drop_duplicates()] #每張表都是一場比賽
inn = []
for i in game:
    for j in i['innAndHalf'].drop_duplicates():
        inns = i.loc[i.innAndHalf == j]
        inn.append(inns)
# 在切表，變一局一局
for i in inn:
    new = pd.DataFrame([['','','']]).reindex(columns = ['一壘跑者','二壘跑者','三壘跑者'])
    news = new.append(i.iloc[:-1,21:24])
    news.reset_index(inplace=True, drop=True)
    i.reset_index(inplace=True, drop=True)
    i.iloc[:,21:24] = news

In [69]:
# 合併表格
newTable = inn[0]
for i in inn[1:]:
    newTable = pd.concat([newTable,i],ignore_index=True)

In [70]:
# 切表，一個投手一張表
newPitcher = [newTable.loc[newTable.投手 == x] for x in newTable['投手'].drop_duplicates()]

In [71]:
newPitcher[0]

,game_id,No.,總球數,主隊得分,客隊得分,局數,Inning Half,投手,捕手,投手球數,好球數,壞球數,出局數,球速,球種,打者,揮棒,進壘點X,進壘點Y,好壞球,事件,一壘跑者,二壘跑者,三壘跑者,打點,好球,壞球,Mark In,打者用手,innAndHalf
0,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,1,1,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,1,1.0,0.0,0.0,140.0,速球,統一陳傑憲,未出棒,100.650002,111.949997,好球,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,L,1top
1,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,2,2,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,2,2.0,0.0,0.0,139.0,速球,統一陳傑憲,出棒,58.450001,101.750000,好球,NaN,NaN,NaN,NaN,NaN,2.0,0.0,7.0,L,1top
2,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,3,3,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,3,3.0,0.0,1.0,147.0,速球,統一陳傑憲,出棒,56.049999,104.150002,好球,NaN,NaN,NaN,NaN,NaN,3.0,0.0,33.0,L,1top
3,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,4,4,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,4,4.0,0.0,1.0,145.0,速球,統一林靖凱,未出棒,49.400002,118.000000,好球,NaN,NaN,NaN,NaN,NaN,1.0,0.0,66.0,R,1top
4,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,5,5,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,5,5.0,0.0,1.0,147.0,速球,統一林靖凱,未出棒,64.449997,114.400002,好球,NaN,NaN,NaN,NaN,NaN,2.0,0.0,86.0,R,1top
5,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,6,6,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,6,6.0,0.0,1.0,143.0,速球,統一林靖凱,出棒,103.050003,70.449997,好球,NaN,NaN,NaN,NaN,NaN,2.0,0.0,108.0,R,1top
6,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,7,7,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,7,6.0,1.0,1.0,131.0,滑球,統一林靖凱,未出棒,104.849998,158.899994,壞球,NaN,NaN,NaN,NaN,NaN,2.0,1.0,135.0,R,1top
7,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,8,8,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,8,6.0,2.0,1.0,131.0,指叉球,統一林靖凱,未出棒,90.349998,157.699997,壞球,NaN,NaN,NaN,NaN,NaN,2.0,2.0,163.0,R,1top
8,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,9,9,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,9,7.0,2.0,2.0,126.0,滑球,統一林靖凱,出棒,45.799999,110.150002,好球,NaN,NaN,NaN,NaN,NaN,3.0,2.0,190.0,R,1top
9,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,10,10,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,10,8.0,2.0,2.0,147.0,速球,統一林安可,未出棒,107.849998,101.150002,好球,NaN,NaN,NaN,NaN,NaN,1.0,0.0,229.0,L,1top


In [72]:
# 得點圈有人成績
t  = newPitcher[0]
pointCircle = t[(t.loc[:,'二壘跑者'].isnull() != True) | (t.loc[:,'三壘跑者'].isnull() != True)]
pointCircle

,game_id,No.,總球數,主隊得分,客隊得分,局數,Inning Half,投手,捕手,投手球數,好球數,壞球數,出局數,球速,球種,打者,揮棒,進壘點X,進壘點Y,好壞球,事件,一壘跑者,二壘跑者,三壘跑者,打點,好球,壞球,Mark In,打者用手,innAndHalf
21,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,22,22,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,22,14.0,8.0,2.0,NaN,速球,統一陳鏞基,未出棒,100.000000,46.349998,壞球,NaN,統一蘇智傑,統一林安可,NaN,NaN,0.0,1.0,579.0,R,1top
22,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,23,23,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,23,15.0,8.0,2.0,NaN,速球,統一陳鏞基,出棒,94.000000,107.750000,好球,NaN,統一蘇智傑,統一林安可,NaN,NaN,1.0,1.0,602.0,R,1top
23,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,24,24,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,24,15.0,9.0,2.0,NaN,無法判定,統一陳鏞基,未出棒,92.199997,155.300003,壞球,NaN,統一蘇智傑,統一林安可,NaN,NaN,1.0,2.0,646.0,R,1top
24,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,25,25,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,25,15.0,10.0,2.0,NaN,無法判定,統一陳鏞基,未出棒,113.250000,142.050003,壞球,NaN,統一蘇智傑,統一林安可,NaN,NaN,1.0,3.0,693.0,R,1top
25,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,26,26,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,26,15.0,11.0,2.0,NaN,速球,統一陳鏞基,未出棒,53.650002,49.349998,壞球,NaN,統一蘇智傑,統一林安可,NaN,NaN,1.0,4.0,721.0,R,1top
26,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,27,27,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,27,15.0,12.0,2.0,NaN,速球,統一高國慶,未出棒,91.599998,45.150002,壞球,NaN,統一陳鏞基,統一蘇智傑,統一林安可,NaN,0.0,1.0,766.0,R,1top
27,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,28,28,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,28,16.0,12.0,2.0,NaN,速球,統一高國慶,未出棒,94.599998,116.199997,好球,NaN,統一陳鏞基,統一蘇智傑,統一林安可,NaN,1.0,1.0,806.0,R,1top
28,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,29,29,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,29,17.0,12.0,2.0,NaN,速球,統一高國慶,出棒,89.199997,106.550003,好球,NaN,統一陳鏞基,統一蘇智傑,統一林安可,NaN,2.0,1.0,834.0,R,1top
29,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,30,30,0.0,0.0,1.0,top,兄弟米蘭達,兄弟高宇杰,30,18.0,12.0,3.0,NaN,速球,統一高國慶,未出棒,100.000000,114.400002,好球,NaN,統一陳鏞基,統一蘇智傑,統一林安可,NaN,3.0,1.0,864.0,R,1top
52,20200412-統一vs兄弟-54費爾本23劉軒荅29王玉譜40黃竣彥60鄭鈞仁12陳韻文...,53,52,0.0,1.0,2.0,top,兄弟米蘭達,兄弟高宇杰,52,31.0,21.0,1.0,133.0,滑球,統一林靖凱,未出棒,99.400002,95.750000,好球,NaN,統一陳傑憲,NaN,統一江亮緯,NaN,1.0,0.0,2056.0,R,2top


In [ ]:
# 好球數、壞球數、好壞球比、得點圈被打擊率、被全壘打率被長打率
